In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2

In [6]:
import os 
import torch
import numpy as np

from torch.utils.tensorboard import SummaryWriter
from os.path import join as pjoin
from torch.distributions import Categorical
import json
import clip

import options.option_transformer as option_trans
import models.vqvae as vqvae
import utils.utils_model as utils_model
import utils.eval_trans as eval_trans
from dataset import dataset_TM_train
from dataset import dataset_TM_eval
from dataset import dataset_tokenize
import models.t2m_trans as trans
from options.get_eval_option import get_opt
from models.evaluator_wrapper import EvaluatorModelWrapper
import warnings
warnings.filterwarnings('ignore')
from exit.VQVAE_transformer import VQVAETransformer
from exit.motiontransformer import MotionTransformerOnly2, generate_src_mask

In [3]:
class Temp:
    def __init__(self):
        print('mock:: opt')
args = Temp()
args.dataname = args.dataset_name = 'kit'
args.down_t = 2

train_loader_token = dataset_tokenize.DATALoader(args.dataname, 1, unit_length=2**args.down_t)

from utils.word_vectorizer import WordVectorizer
w_vectorizer = WordVectorizer('./glove', 'our_vab')
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

dataset_opt_path = 'checkpoints/kit/Comp_v6_KLD005/opt.txt' if args.dataname == 'kit' else 'checkpoints/t2m/Comp_v6_KLD005/opt.txt'

wrapper_opt = get_opt(dataset_opt_path, torch.device('cuda'))
eval_wrapper = EvaluatorModelWrapper(wrapper_opt)

mock:: opt


100%|██████████| 300/300 [00:00<00:00, 1830.28it/s]


Pointer Pointing at 0
Reading checkpoints/kit/Comp_v6_KLD005/opt.txt
Loading Evaluation Model Wrapper (Epoch 30) Completed!!


In [4]:
##### ---- Network ---- #####
clip_model, clip_preprocess = clip.load("ViT-B/32", device=torch.device('cuda'), jit=False)  # Must set jit=False for training
clip.model.convert_weights(clip_model)  # Actually this line is unnecessary since clip by default already on float16
clip_model.eval()
for p in clip_model.parameters():
    p.requires_grad = False

In [10]:
args.nb_code = 512 # 8192 # 
args.code_dim = 512 # 32 # 
args.output_emb_width = 512
args.down_t = 2
args.stride_t = 2
args.width = 512
args.depth = 3
args.dilation_growth_rate = 3
args.vq_act = 'relu'
args.vq_norm = None
args.quantizer = 'ema_reset'
args.mu = 0.99
args.embed_dim_gpt = 1024
args.clip_dim = 512
args.block_size = 51
args.num_layers = 9
args.n_head_gpt = 16
args.drop_out_rate = 0.1
args.ff_rate = 4
trans_encoder = trans.Text2Motion_Transformer(num_vq=args.nb_code, 
                                            embed_dim=args.embed_dim_gpt, 
                                            clip_dim=args.clip_dim, 
                                            block_size=args.block_size, 
                                            num_layers=args.num_layers, 
                                            n_head=args.n_head_gpt, 
                                            drop_out_rate=args.drop_out_rate, 
                                            fc_rate=args.ff_rate)


net = VQVAETransformer(args)
args.resume_pth = '/home/epinyoan/git/MaskText2Motion/T2M-GPT/output/VQVAE/net_last.pth'
# ckpt = torch.load(args.resume_pth, map_location='cpu')
# net.load_state_dict(ckpt['net'], strict=True)
net.eval()
net.cuda()
trans_encoder.train()
trans_encoder.cuda()
''

RuntimeError: Error(s) in loading state_dict for VQVAETransformer:
	Missing key(s) in state_dict: "encoder.sequence_embedding", "encoder.joint_embed.weight", "encoder.joint_embed.bias", "encoder.temporal_blocks.0.norm1.weight", "encoder.temporal_blocks.0.norm1.bias", "encoder.temporal_blocks.0.attn.qkv.weight", "encoder.temporal_blocks.0.attn.qkv.bias", "encoder.temporal_blocks.0.attn.proj.weight", "encoder.temporal_blocks.0.attn.proj.bias", "encoder.temporal_blocks.0.norm2.weight", "encoder.temporal_blocks.0.norm2.bias", "encoder.temporal_blocks.0.mlp.fc1.weight", "encoder.temporal_blocks.0.mlp.fc1.bias", "encoder.temporal_blocks.0.mlp.fc2.weight", "encoder.temporal_blocks.0.mlp.fc2.bias", "encoder.temporal_blocks.1.norm1.weight", "encoder.temporal_blocks.1.norm1.bias", "encoder.temporal_blocks.1.attn.qkv.weight", "encoder.temporal_blocks.1.attn.qkv.bias", "encoder.temporal_blocks.1.attn.proj.weight", "encoder.temporal_blocks.1.attn.proj.bias", "encoder.temporal_blocks.1.norm2.weight"

### VQVAE encoder index precomputed

In [9]:
for batch in train_loader_token:
    pose, name = batch
    bs, seq = pose.shape[0], pose.shape[1]

    pose = pose.cuda().float() # bs, nb_joints, joints_dim, seq_len
    target = net.encode(pose)
    target = target.cpu().numpy()
    break

AttributeError: 'VQVAETransformer' object has no attribute 'encode'

### Train

In [6]:
args.batch_size = 1024
num_workers = 8
args.vq_name = 'VQVAE'
train_loader = dataset_TM_train.DATALoader(args.dataname, args.batch_size, args.nb_code, args.vq_name, unit_length=2**args.down_t, num_workers=num_workers)
train_loader_iter = dataset_TM_train.cycle(train_loader)

100%|██████████| 4888/4888 [00:01<00:00, 2952.82it/s]


In [1]:
######## count used indices
# all_indices = None
# for batch in train_loader:
#     clip_text, m_tokens, m_tokens_len = batch
#     if all_indices is None:
#         all_indices = m_tokens
#     else:
#         all_indices = torch.cat([all_indices, m_tokens])
# a = all_indices.view(-1)
# condition = a < 512
# c = torch.bincount(a[condition])
# (c==0).sum()

In [18]:
args.pkeep = .5
batch = next(train_loader_iter)
clip_text, m_tokens, m_tokens_len = batch
m_tokens, m_tokens_len = m_tokens.cuda(), m_tokens_len.cuda()
bs = m_tokens.shape[0]
target = m_tokens    # (bs, 26)
target = target.cuda()

text = clip.tokenize(clip_text, truncate=True).cuda()
    
feat_clip_text = clip_model.encode_text(text).float()

input_index = target[:,:-1]

if args.pkeep == -1:
    proba = np.random.rand(1)[0]
    mask = torch.bernoulli(proba * torch.ones(input_index.shape,
                                                        device=input_index.device))
else:
    mask = torch.bernoulli(args.pkeep * torch.ones(input_index.shape,
                                                        device=input_index.device))
mask = mask.round().to(dtype=torch.int64)
r_indices = torch.randint_like(input_index, args.nb_code)
a_indices = mask*input_index+(1-mask)*r_indices

cls_pred = trans_encoder(a_indices, feat_clip_text)
cls_pred = cls_pred.contiguous()
cls_pred.shape

torch.Size([2, 51, 513])

In [19]:
args.if_maxtest = False

right_num = 0
loss_ce = torch.nn.CrossEntropyLoss()
loss_cls = 0.0
for i in range(bs):
    # loss function     (26), (26, 513)
    loss_cls += loss_ce(cls_pred[i][:m_tokens_len[i] + 1], target[i][:m_tokens_len[i] + 1]) / bs

    # Accuracy
    probs = torch.softmax(cls_pred[i][:m_tokens_len[i] + 1], dim=-1)

    if args.if_maxtest:
        _, cls_pred_index = torch.max(probs, dim=-1)
    else:
        dist = Categorical(probs)
        cls_pred_index = dist.sample()
    right_num += (cls_pred_index.flatten(0) == target[i][:m_tokens_len[i] + 1].flatten(0)).sum().item()

### Eval

In [6]:
val_loader = dataset_TM_eval.DATALoader(args.dataname, False, 32, w_vectorizer)

100%|██████████| 300/300 [00:00<00:00, 3027.56it/s]

Pointer Pointing at 0


In [7]:
class LoggerWriterMock:
    def __init__(self):
        self.info
    def info(self, *args):
        print(*args)
    def add_scalar(self, *args):
        print(*args)
    def add_video(self, *args):
        print(*args)
logger = LoggerWriterMock()
logger.info('test')
writer = LoggerWriterMock()
writer.add_scalar('./Test/FID', 'fid', 'nb_iter')

test
./Test/FID fid nb_iter


In [8]:
args.resume_trans = 'output/1_TRANS/net_best_fid.pth'
ckpt = torch.load(args.resume_trans, map_location='cpu')
trans_encoder.load_state_dict(ckpt['trans'], strict=True)

<All keys matched successfully>

In [10]:
### Run only one batch ###
batch = next(iter(val_loader))
word_embeddings, pos_one_hots, clip_text, sent_len, pose, m_length, token, name = batch
bs, seq = pose.shape[:2]
num_joints = 21 if pose.shape[-1] == 251 else 22

text = clip.tokenize(clip_text, truncate=True).cuda()

feat_clip_text = clip_model.encode_text(text).float()
pred_pose_eval = torch.zeros((bs, seq, pose.shape[-1])).cuda()
pred_len = torch.ones(bs).long()
for k in range(bs):
    index_motion = trans_encoder.sample(feat_clip_text[k:k+1], False)
    pred_pose = net.forward_decoder(index_motion)
    cur_len = pred_pose.shape[1]

In [ ]:
### Run full eval (slow) ###
args.out_dir = './'
best_fid, best_iter, best_div, best_top1, best_top2, best_top3, best_matching, writer, logger = \
    eval_trans.evaluation_transformer(args.out_dir, 
        val_loader, 
        net, 
        trans_encoder, 
        logger, 
        writer, 
        0, 
        best_fid=1000, 
        best_iter=0, 
        best_div=100, 
        best_top1=0, 
        best_top2=0, 
        best_top3=0, 
        best_matching=100, 
        clip_model=clip_model, 
        eval_wrapper=eval_wrapper)